In [1]:
import pandas as pd
import plotly.express as px
from internproject.constants import ROOT_DATA_DIR

add_lp = pd.read_parquet(ROOT_DATA_DIR / "stETH_ETH_add_lp.parquet")  # more details on the add liquidity events
remove_lp = pd.read_parquet(
    ROOT_DATA_DIR / "stETH_ETH_remove_lp.parquet"
)  # more details on the remove liquidity events
lp_transfers = pd.read_parquet(ROOT_DATA_DIR / "stETH_ETH_lp_transfers.parquet")

In [2]:
null_address = "0x0000000000000000000000000000000000000000"

In [8]:
lp_transfers

,from,to,value,event,block,transaction_index,log_index,hash,timestamp
timestamp,,,,,,,,,
2021-01-05 00:26:18,0x0000000000000000000000000000000000000000,0x7EeAC6CDdbd1D0B8aF061742D41877D7F707289a,0.000000,Transfer,11591275,88,148,0x4fdd5de055a85451535ae48f21353f922554d9416903...,2021-01-05 00:26:18
2021-01-05 11:26:21,0x0000000000000000000000000000000000000000,0x6ab615CF8deCFc488186E54066Fc10589C9293A3,0.160000,Transfer,11594223,141,250,0x16d97e10e582735b4731aaec2823f781edb3355fa9b2...,2021-01-05 11:26:21
2021-01-05 13:35:18,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,69.999492,Transfer,11594829,94,97,0x8eca09399384c600131f1f9e1d931246e9129f2e1e11...,2021-01-05 13:35:18
2021-01-05 13:36:06,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,0x182B723a58739a9c974cFDB385ceaDb237453c28,69.999492,Transfer,11594833,50,43,0xacd73e0a596914d36650b57a0bf264cc8a8b63e053a8...,2021-01-05 13:36:06
2021-01-05 13:48:47,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,1999.989998,Transfer,11594889,122,123,0x6c6cb813a1cfee7cb6bb241b74a961ede8b70eed5ae3...,2021-01-05 13:48:47
...,...,...,...,...,...,...,...,...,...
2023-06-21 00:00:11,0x99ac10631F69C753DDb595D074422a0922D9056B,0x182B723a58739a9c974cFDB385ceaDb237453c28,0.003230,Transfer,17524329,349,316,0xfa9e275dc3d9f53a90c0c817cb16fa6f328acb696b49...,2023-06-21 00:00:11
2023-06-21 00:00:11,0x182B723a58739a9c974cFDB385ceaDb237453c28,0x989AEb4d175e16225E39E87d0D97A3360524AD80,0.003230,Transfer,17524329,349,318,0xfa9e275dc3d9f53a90c0c817cb16fa6f328acb696b49...,2023-06-21 00:00:11
2023-06-21 00:00:11,0x989AEb4d175e16225E39E87d0D97A3360524AD80,0xF403C135812408BFbE8713b5A23a04b3D48AAE31,0.003230,Transfer,17524329,349,321,0xfa9e275dc3d9f53a90c0c817cb16fa6f328acb696b49...,2023-06-21 00:00:11


### Understanding lp transfers

1. The "value" column in `lp_transfers` refers to the quantity of lp tokens
2. If the null address is in the "from" column, then it is adding liquidity to the pool. 
3. If the null address is in the "to" column, then it is removeing liquidity from the pool.
4. It is safe to ignore the other transfers

In [3]:
add_liquidity_transfers = lp_transfers[lp_transfers["from"] == null_address]
add_liquidity_transfers

,from,to,value,event,block,transaction_index,log_index,hash,timestamp
timestamp,,,,,,,,,
2021-01-05 00:26:18,0x0000000000000000000000000000000000000000,0x7EeAC6CDdbd1D0B8aF061742D41877D7F707289a,0.000000,Transfer,11591275,88,148,0x4fdd5de055a85451535ae48f21353f922554d9416903...,2021-01-05 00:26:18
2021-01-05 11:26:21,0x0000000000000000000000000000000000000000,0x6ab615CF8deCFc488186E54066Fc10589C9293A3,0.160000,Transfer,11594223,141,250,0x16d97e10e582735b4731aaec2823f781edb3355fa9b2...,2021-01-05 11:26:21
2021-01-05 13:35:18,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,69.999492,Transfer,11594829,94,97,0x8eca09399384c600131f1f9e1d931246e9129f2e1e11...,2021-01-05 13:35:18
2021-01-05 13:48:47,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,1999.989998,Transfer,11594889,122,123,0x6c6cb813a1cfee7cb6bb241b74a961ede8b70eed5ae3...,2021-01-05 13:48:47
2021-01-05 14:32:47,0x0000000000000000000000000000000000000000,0xb084F0AdB86BF6B001E540Ac05439D98A499ce1D,0.999754,Transfer,11595091,101,132,0xa3ffd6f2c890037b38c11d64e34e991fa1f31318e873...,2021-01-05 14:32:47
...,...,...,...,...,...,...,...,...,...
2023-06-20 13:17:47,0x0000000000000000000000000000000000000000,0x60F727BdeAD2ce49B00f2A2133Fc707b931D130B,0.000930,Transfer,17521155,157,419,0xf82ec22d0978797d3bb8428986d19bded9cac2cec209...,2023-06-20 13:17:47
2023-06-20 14:07:59,0x0000000000000000000000000000000000000000,0x93621DCA56fE26Cdee86e4F6B18E116e9758Ff11,0.000930,Transfer,17521403,16,113,0x1ca5708625a94e5a4365062ded1116c9d08b501f2d96...,2023-06-20 14:07:59
2023-06-20 14:52:23,0x0000000000000000000000000000000000000000,0x64bB6967b85F7084C367774bc840b4a4c4Afcd8E,0.005861,Transfer,17521623,50,55,0xeb436efb3b0de1c2adb18fa01f58b5d99ffebf63ff8c...,2023-06-20 14:52:23


In [4]:
withdraw_liquidity_transfers = lp_transfers[lp_transfers["to"] == null_address]
withdraw_liquidity_transfers

,from,to,value,event,block,transaction_index,log_index,hash,timestamp
timestamp,,,,,,,,,
2021-01-05 17:12:33,0x3a63aA8AE11Ecaa82fC352d0390634E8030B2ffd,0x0000000000000000000000000000000000000000,2.000945,Transfer,11595786,133,295,0xbd58a23237562e0332aaa7a7668f8dd71e4aecddc95d...,2021-01-05 17:12:33
2021-01-05 17:46:38,0xaF297deC752c909092A117A932A8cA4AaaFF9795,0x0000000000000000000000000000000000000000,499.501433,Transfer,11595955,11,5,0xad790ed2cb537281ae0c7eaf1558b1494b869e7f7e1c...,2021-01-05 17:46:38
2021-01-05 21:03:54,0xb084F0AdB86BF6B001E540Ac05439D98A499ce1D,0x0000000000000000000000000000000000000000,0.999754,Transfer,11596894,75,96,0xc485ceb5703ddaf5393049b8fb71a06b20ee0c0b4ced...,2021-01-05 21:03:54
2021-01-06 03:36:23,0x89B8B20AE90328692cD367f75aaFadF55fd33E8B,0x0000000000000000000000000000000000000000,10.037649,Transfer,11598604,77,130,0xbebdcd6fa25430bea017919704fd3dcd12f55ef0f99b...,2021-01-06 03:36:23
2021-01-06 12:17:18,0xdE3E412d4fe3c9d90ac74d0A9B064951B39EEae4,0x0000000000000000000000000000000000000000,1.002055,Transfer,11600957,56,66,0x5a33f4f9e2e2a55f09141909f75b1b308d092183681c...,2021-01-06 12:17:18
...,...,...,...,...,...,...,...,...,...
2023-06-20 22:25:35,0xEa508F82728927454bd3ce853171b0e2705880D4,0x0000000000000000000000000000000000000000,0.000000,Transfer,17523858,45,95,0x6188d0177bb9a8a4a3bb5feb560477d6ec4642c9fde0...,2023-06-20 22:25:35
2023-06-20 22:33:11,0x46D5F548E02fdee3ea7E4168B7cF896429fE6fe7,0x0000000000000000000000000000000000000000,65.033669,Transfer,17523896,78,240,0x113260903e90d948871a8a0af2f03156c17781a455ea...,2023-06-20 22:33:11
2023-06-20 22:40:35,0x5aD15EBa9D4E9351414084Eb1dd1EACc2068A4A7,0x0000000000000000000000000000000000000000,51.167100,Transfer,17523933,42,101,0x39af71a3003ae573c6739b9679bede548ea7cc53a6b0...,2023-06-20 22:40:35


In [5]:
lp_transfers

,from,to,value,event,block,transaction_index,log_index,hash,timestamp
timestamp,,,,,,,,,
2021-01-05 00:26:18,0x0000000000000000000000000000000000000000,0x7EeAC6CDdbd1D0B8aF061742D41877D7F707289a,0.000000,Transfer,11591275,88,148,0x4fdd5de055a85451535ae48f21353f922554d9416903...,2021-01-05 00:26:18
2021-01-05 11:26:21,0x0000000000000000000000000000000000000000,0x6ab615CF8deCFc488186E54066Fc10589C9293A3,0.160000,Transfer,11594223,141,250,0x16d97e10e582735b4731aaec2823f781edb3355fa9b2...,2021-01-05 11:26:21
2021-01-05 13:35:18,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,69.999492,Transfer,11594829,94,97,0x8eca09399384c600131f1f9e1d931246e9129f2e1e11...,2021-01-05 13:35:18
2021-01-05 13:36:06,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,0x182B723a58739a9c974cFDB385ceaDb237453c28,69.999492,Transfer,11594833,50,43,0xacd73e0a596914d36650b57a0bf264cc8a8b63e053a8...,2021-01-05 13:36:06
2021-01-05 13:48:47,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,1999.989998,Transfer,11594889,122,123,0x6c6cb813a1cfee7cb6bb241b74a961ede8b70eed5ae3...,2021-01-05 13:48:47
...,...,...,...,...,...,...,...,...,...
2023-06-21 00:00:11,0x99ac10631F69C753DDb595D074422a0922D9056B,0x182B723a58739a9c974cFDB385ceaDb237453c28,0.003230,Transfer,17524329,349,316,0xfa9e275dc3d9f53a90c0c817cb16fa6f328acb696b49...,2023-06-21 00:00:11
2023-06-21 00:00:11,0x182B723a58739a9c974cFDB385ceaDb237453c28,0x989AEb4d175e16225E39E87d0D97A3360524AD80,0.003230,Transfer,17524329,349,318,0xfa9e275dc3d9f53a90c0c817cb16fa6f328acb696b49...,2023-06-21 00:00:11
2023-06-21 00:00:11,0x989AEb4d175e16225E39E87d0D97A3360524AD80,0xF403C135812408BFbE8713b5A23a04b3D48AAE31,0.003230,Transfer,17524329,349,321,0xfa9e275dc3d9f53a90c0c817cb16fa6f328acb696b49...,2023-06-21 00:00:11


### when did this person enter and leave

In [6]:
example_person = "0x80ee7c0e1e59929823eD9B22e34538b226967109"
add_liquidity_transfers[add_liquidity_transfers["to"] == example_person]

,from,to,value,event,block,transaction_index,log_index,hash,timestamp
timestamp,,,,,,,,,
2021-01-12 22:16:32,0x0000000000000000000000000000000000000000,0x80ee7c0e1e59929823eD9B22e34538b226967109,1.999396,Transfer,11642841,30,51,0xb03636820959b9641a5c4c39676967180d7629a9bc30...,2021-01-12 22:16:32
2021-01-12 22:31:11,0x0000000000000000000000000000000000000000,0x80ee7c0e1e59929823eD9B22e34538b226967109,0.999698,Transfer,11642902,120,187,0xcf8723e0451638d3f68a12395facc8c0f1ed2178a90f...,2021-01-12 22:31:11


In [7]:
withdraw_liquidity_transfers[withdraw_liquidity_transfers["from"] == example_person]

,from,to,value,event,block,transaction_index,log_index,hash,timestamp
timestamp,,,,,,,,,
2021-01-14 13:13:07,0x80ee7c0e1e59929823eD9B22e34538b226967109,0x0000000000000000000000000000000000000000,2.999095,Transfer,11653414,210,243,0x642a0a0273035551b6fa585c8615e129bc05772c9277...,2021-01-14 13:13:07


This person entered twice at 2021-01-12 and then left at 2021-01-14. So they were only in for 3 days.

### Open Problems

Here are some problems you could look at. Remember that the focus is figuring out how changing incentive APR affacts LPs entering and leaving the pool

1. How does the amount of ETH value added influence the duration of a person's stay in the pool? (does this relationship change over time?)
2. Among those who fully exited the pool, what was the difference between the starting and ending Annual Percentage Rate (APR)?
3. What was the average duration of people's participation in the pool?
4. How frequently did people perform actions such as adding liquidity or withdrawing liquidity? Did they break up these actions into smaller increments?
5. Can the participants be clustered into meaningful groups based on their LP sizes and durations?
6. Are there any notable thresholds or breakpoints at which people exhibit different behaviors? eg. after a person has been in a pool for > 1 month they almost never leave
7. It is important to pay special attention to participants with large LPs. How do the people who have added and withdrawn large amounts of liquidity differ?